<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Part-1:" data-toc-modified-id="Part-1:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Part 1:</a></span><ul class="toc-item"><li><span><a href="#To-do:-drop-columns-that-don't-contain-useful-data:" data-toc-modified-id="To-do:-drop-columns-that-don't-contain-useful-data:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>To do: drop columns that don't contain useful data:</a></span></li></ul></li></ul></div>

# Data Question 4: Nashville COVID-19 Violations  
  
In this project, you will explore COVID-19 violations reported to hubNashville, Metro Nashville government's comprehensive customer service system.  
  
### Part 1:  
  
Gather the dataset of hubNashville requests from https://data.nashville.gov/Public-Services/hubNashville-311-Service-Requests/7qhx-rexh. For this project, you should look at requests with Request Type of "COVID-19" and Subrequest Type of "COVID-19 Violations". Explore this dataset, looking at when and where these violations occurred.  

In [1]:
import pandas as pd
import numpy as np
import re
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline

import requests
import json
from urllib.request import urlretrieve
from bs4 import BeautifulSoup as BS
from collections import defaultdict
from IPython.core.display import HTML

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# # Set API url
# url = 'https://data.nashville.gov/resource/7qhx-rexh.json'

# # Create request
# response = requests.get(url)

In [4]:
# response.json()

In [5]:
# type(response.json()) # a list of dictionaries

In [6]:
# covid_v = pd.read_json(response.text) # turns json into python dataframe; can also plug-in json dataframe

In [7]:
# covid_v

In [8]:
# sorted(covid_v.columns)

In [9]:
# Set API url
url = 'https://data.nashville.gov/resource/7qhx-rexh.csv'

# Create request
r = requests.get(url)

In [10]:
violations = pd.read_csv(StringIO(r.text))

In [11]:
violations

,case_number,status,case_request,case_subrequest,additional_subrequest,date_time_opened,date_time_closed,case_origin,oem_id,contact_type,preferred_language,state_issue,parent_case,closed_when_created,incident_address,incident_city,incident_council_district,incident_zip_code,latitude,longitude,mapped_location
0,353977,Closed,COVID-19,COVID-19 - Submit Input about The Plan to Reop...,COVID-19 - Submit Input about The Plan to Reop...,2020-04-28T14:33:22.000,2020-04-28T14:33:22.000,hubNashville Community,NaN,NaN,NaN,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,367021,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2020-05-13T21:00:52.000,2020-05-13T21:00:52.000,Phone,NaN,NaN,NaN,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,419194,Closed,"Streets, Roads & Sidewalks",Remove debris in roadway,Roadway,2020-07-17T15:21:03.000,2020-07-28T16:06:19.000,hubNashville Community,NaN,NaN,NaN,False,NaN,False,Lafayette St & Claiborne St,NASHVILLE,17.0,37210.0,36.148412,-86.761179,"(36.1484116, -86.761179)"
3,366600,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2020-05-13T16:04:14.000,2020-05-13T16:04:14.000,Phone,NaN,NaN,NaN,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,408945,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2020-07-06T17:57:28.000,2020-07-06T17:57:28.000,Phone,NaN,NaN,NaN,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,313963,Closed,"Streets, Roads & Sidewalks",Request New Sign,Request New Sign,2020-02-26T17:16:30.000,2020-04-20T16:07:17.000,Phone,NaN,NaN,NaN,False,NaN,False,Sawyer Brown Rd & Todd Preis Dr,NASHVILLE,22.0,37221.0,36.071177,-86.947000,"(36.071176500000014, -86.94699975000002)"
996,366810,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2020-05-13T18:06:46.000,2020-05-13T18:06:46.000,Phone,NaN,NaN,NaN,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,379022,Closed,"Trash, Recycling & Litter",Cart Service,Recycling - Report a Stolen Cart,2020-05-28T20:42:35.000,2020-06-23T16:20:50.000,Phone,NaN,NaN,NaN,False,NaN,False,754 Spence Enclave Ln,NASHVILLE,15.0,37210.0,36.152286,-86.719218,"(36.1522864, -86.71921809999999)"
998,384000,Closed,Property Violations,Cars Parked on Lawn,Cars Parked on Lawn,2020-06-04T12:03:41.000,2020-06-06T07:20:22.000,hubNashville Community,NaN,NaN,NaN,False,NaN,False,764 Fitzpatrick Rd,NASHVILLE,13.0,37214.0,36.150990,-86.631656,"(36.15099021160178, -86.6316556)"


In [12]:
violations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   case_number                1000 non-null   int64  
 1   status                     1000 non-null   object 
 2   case_request               1000 non-null   object 
 3   case_subrequest            1000 non-null   object 
 4   additional_subrequest      995 non-null    object 
 5   date_time_opened           1000 non-null   object 
 6   date_time_closed           982 non-null    object 
 7   case_origin                1000 non-null   object 
 8   oem_id                     33 non-null     object 
 9   contact_type               7 non-null      object 
 10  preferred_language         0 non-null      float64
 11  state_issue                1000 non-null   bool   
 12  parent_case                10 non-null     float64
 13  closed_when_created        1000 non-null   bool  

In [13]:
covid_only = [violations['case_request'] == 'COVID-19']
covid_only

[0       True
 1      False
 2      False
 3      False
 4      False
        ...  
 995    False
 996    False
 997    False
 998    False
 999    False
 Name: case_request, Length: 1000, dtype: bool]

We're missing data. Should have 21 columns and 479K rows, but we're only getting 1000...so we have to read in the csv file.

In [14]:
violations = pd.read_csv('../data/hubNashville__311__Service_Requests.csv', low_memory=False)

In [15]:
violations

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,Preferred Language,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,113016,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Request an Additional Cart,10/30/2018 01:47:00 PM,03/05/2019 09:35:00 PM,Phone,NaN,NaN,NaN,False,NaN,False,2009 Upland Dr,NASHVILLE,7.0,37216,36.198638,-86.706659,"(36.1986378, -86.7066586)"
1,53502,Closed,Property Violations,Residential Complaint,Residential Complaint,03/13/2018 09:34:50 PM,04/06/2018 01:13:32 PM,Phone,NaN,NaN,NaN,False,NaN,False,631 Murfreesboro Pike,NASHVILLE,17.0,37210,36.140070,-86.738360,"(36.14007, -86.73836)"
2,119344,Closed,"Trash, Recycling & Litter",Damage to Property,Trash Service,11/27/2018 02:20:00 PM,11/29/2018 04:04:00 AM,Phone,NaN,NaN,NaN,False,NaN,True,"597 Westchester Dr, Nashville, TN 37115, Unite...",MADISON,8.0,37115,36.263020,-86.738630,"(36.26302, -86.73863)"
3,127752,Closed,"Trash, Recycling & Litter",New Service/Activate Service,New Construction Residential Recycle,01/03/2019 06:57:00 PM,01/09/2019 10:09:00 AM,Phone,NaN,NaN,NaN,False,NaN,False,Ruffian Way,NaN,NaN,NaN,38.097609,-84.554097,"(38.0976091, -84.5540971)"
4,113477,Closed,"Trash, Recycling & Litter",Backdoor Trash or Recycle Waivers,Request Recycling Disability Waiver,10/31/2018 12:20:00 PM,10/31/2018 04:10:00 PM,Phone,NaN,NaN,NaN,False,NaN,True,"2326 Smith Springs Rd, Nashville, TN 37217, Un...",NASHVILLE,29.0,37217,36.096460,-86.637440,"(36.09646, -86.63744)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479301,131156,In Progress,"Streets, Roads & Sidewalks",Traffic Light Timing,Traffic Light Timing,01/14/2019 05:17:51 PM,NaN,Phone,NaN,NaN,NaN,False,NaN,False,Charlotte Ave & City Ave,NASHVILLE,21.0,37209,36.153360,-86.820846,"(36.1533597, -86.8208464)"
479302,112617,In Progress,"Streets, Roads & Sidewalks",Obstruction of right of way,Obstruction of right of way,10/28/2018 09:16:04 PM,NaN,hubNashville Community,NaN,NaN,NaN,False,NaN,False,504 Hill Rd,NASHVILLE,4.0,37220,36.047498,-86.749553,"(36.0474983, -86.74955339999997)"
479303,114914,In Progress,"Streets, Roads & Sidewalks",Traffic Engineering,Intersection Review,11/06/2018 10:45:40 PM,NaN,Phone,NaN,NaN,NaN,False,NaN,False,28th Ave N & Meharry Blvd,NASHVILLE,21.0,37209,36.166576,-86.820434,"(36.1665756, -86.82043449999998)"
479304,116292,In Progress,"Streets, Roads & Sidewalks",Traffic Engineering,Request Markings,11/13/2018 04:31:57 PM,NaN,Phone,NaN,NaN,NaN,False,NaN,False,2805 Abbott Martin Rd,NASHVILLE,34.0,37215,36.108803,-86.832898,"(36.1088031, -86.83289789999998)"


In [16]:
covid_only = violations[violations['Request Type'] == 'COVID-19']
covid_only

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,Preferred Language,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
1547,444823,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,08/14/2020 02:38:00 AM,08/14/2020 02:37:15 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,2045 Antioch Pike,ANTIOCH,28.0,37013,36.068546,-86.683898,"(36.0685457, -86.6838975)"
2699,396055,Closed,COVID-19,COVID-19 Questions,COVID-19 Questions,06/19/2020 01:23:07 PM,06/30/2020 05:39:23 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3012,395824,Closed,COVID-19,COVID-19 Questions,COVID-19 Questions,06/18/2020 08:59:50 PM,06/19/2020 01:11:15 PM,Phone,NaN,NaN,NaN,False,NaN,False,Charlotte Pike,NASHVILLE,22.0,37209,36.114870,-86.925890,"(36.11487, -86.92589)"
3350,395888,Closed,COVID-19,COVID-19 Guidance for Reopening Businesses,COVID-19 Guidance for Reopening Businesses,06/18/2020 09:58:32 PM,06/23/2020 02:50:03 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3379,479965,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/13/2020 11:48:40 AM,10/13/2020 01:48:08 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,2800 Opryland Dr,NASHVILLE,15.0,37214,36.211269,-86.694302,"(36.211269429469425, -86.6943024067459)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479073,491117,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/29/2020 03:32:36 PM,10/30/2020 02:10:33 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,345 Normandy Cir,NASHVILLE,24.0,37209,36.146383,-86.825807,"(36.14638299999999, -86.825807)"
479079,492065,Closed,COVID-19,COVID-19 Questions,COVID-19 Questions,10/30/2020 04:31:26 PM,10/30/2020 08:15:45 PM,Phone,NaN,NaN,NaN,False,NaN,False,5141 John Hagar Road,HERMITAGE,12.0,37076,36.155110,-86.578910,"(36.15511, -86.57891)"
479105,492410,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/30/2020 09:47:56 PM,11/02/2020 03:50:18 PM,Phone,NaN,NaN,NaN,False,NaN,False,2408 Music Valley Dr,NASHVILLE,15.0,37214,36.218240,-86.697950,"(36.21824, -86.69795)"
479249,491755,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/30/2020 03:50:56 AM,10/30/2020 01:57:59 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,2500 Charlotte Ave,NASHVILLE,21.0,37209,36.155145,-86.814101,"(36.155145311603604, -86.8141005932541)"


In [17]:
covid_only

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,Preferred Language,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
1547,444823,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,08/14/2020 02:38:00 AM,08/14/2020 02:37:15 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,2045 Antioch Pike,ANTIOCH,28.0,37013,36.068546,-86.683898,"(36.0685457, -86.6838975)"
2699,396055,Closed,COVID-19,COVID-19 Questions,COVID-19 Questions,06/19/2020 01:23:07 PM,06/30/2020 05:39:23 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3012,395824,Closed,COVID-19,COVID-19 Questions,COVID-19 Questions,06/18/2020 08:59:50 PM,06/19/2020 01:11:15 PM,Phone,NaN,NaN,NaN,False,NaN,False,Charlotte Pike,NASHVILLE,22.0,37209,36.114870,-86.925890,"(36.11487, -86.92589)"
3350,395888,Closed,COVID-19,COVID-19 Guidance for Reopening Businesses,COVID-19 Guidance for Reopening Businesses,06/18/2020 09:58:32 PM,06/23/2020 02:50:03 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3379,479965,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/13/2020 11:48:40 AM,10/13/2020 01:48:08 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,2800 Opryland Dr,NASHVILLE,15.0,37214,36.211269,-86.694302,"(36.211269429469425, -86.6943024067459)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479073,491117,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/29/2020 03:32:36 PM,10/30/2020 02:10:33 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,345 Normandy Cir,NASHVILLE,24.0,37209,36.146383,-86.825807,"(36.14638299999999, -86.825807)"
479079,492065,Closed,COVID-19,COVID-19 Questions,COVID-19 Questions,10/30/2020 04:31:26 PM,10/30/2020 08:15:45 PM,Phone,NaN,NaN,NaN,False,NaN,False,5141 John Hagar Road,HERMITAGE,12.0,37076,36.155110,-86.578910,"(36.15511, -86.57891)"
479105,492410,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/30/2020 09:47:56 PM,11/02/2020 03:50:18 PM,Phone,NaN,NaN,NaN,False,NaN,False,2408 Music Valley Dr,NASHVILLE,15.0,37214,36.218240,-86.697950,"(36.21824, -86.69795)"
479249,491755,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/30/2020 03:50:56 AM,10/30/2020 01:57:59 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,2500 Charlotte Ave,NASHVILLE,21.0,37209,36.155145,-86.814101,"(36.155145311603604, -86.8141005932541)"


In [18]:
# COVID-19 Violations
covid_only = violations[violations['Subrequest Type'] == 'COVID-19 Violations']
covid_only

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,Preferred Language,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
1547,444823,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,08/14/2020 02:38:00 AM,08/14/2020 02:37:15 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,2045 Antioch Pike,ANTIOCH,28.0,37013,36.068546,-86.683898,"(36.0685457, -86.6838975)"
3379,479965,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/13/2020 11:48:40 AM,10/13/2020 01:48:08 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,2800 Opryland Dr,NASHVILLE,15.0,37214,36.211269,-86.694302,"(36.211269429469425, -86.6943024067459)"
3660,392927,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,06/16/2020 12:35:20 AM,06/17/2020 02:28:19 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,2607 12th Ave S,NASHVILLE,18.0,37204,36.123500,-86.790130,"(36.1235, -86.79013)"
3844,444512,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,08/13/2020 06:50:31 PM,08/14/2020 02:22:11 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,3200 Clarksville Pike,NASHVILLE,2.0,37218,36.194163,-86.831394,"(36.1941628, -86.8313936)"
3876,444440,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,08/13/2020 06:03:09 PM,08/14/2020 02:18:59 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,20 Grundy St,NASHVILLE,19.0,37203,36.158440,-86.788038,"(36.15843971160494, -86.7880380629013)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479064,492450,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/31/2020 12:17:13 AM,11/02/2020 03:40:13 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,210 Anne Dallas Dudley Blvd,NASHVILLE,19.0,37219,36.163000,-86.782000,"(36.163, -86.782)"
479070,491607,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/29/2020 08:50:02 PM,10/30/2020 02:13:33 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,7135 Centennial Pl,NASHVILLE,20.0,37209,36.177188,-86.878230,"(36.1771876, -86.87822969999999)"
479073,491117,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/29/2020 03:32:36 PM,10/30/2020 02:10:33 PM,hubNashville Community,NaN,NaN,NaN,False,NaN,False,345 Normandy Cir,NASHVILLE,24.0,37209,36.146383,-86.825807,"(36.14638299999999, -86.825807)"
479105,492410,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/30/2020 09:47:56 PM,11/02/2020 03:50:18 PM,Phone,NaN,NaN,NaN,False,NaN,False,2408 Music Valley Dr,NASHVILLE,15.0,37214,36.218240,-86.697950,"(36.21824, -86.69795)"


In [19]:
covid_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11632 entries, 1547 to 479249
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Request #                   11632 non-null  int64  
 1   Status                      11632 non-null  object 
 2   Request Type                11632 non-null  object 
 3   Subrequest Type             11632 non-null  object 
 4   Additional Subrequest Type  11623 non-null  object 
 5   Date / Time Opened          11632 non-null  object 
 6   Date / Time Closed          11611 non-null  object 
 7   Request Origin              11632 non-null  object 
 8   System of Origin ID         0 non-null      object 
 9   Contact Type                46 non-null     object 
 10  Preferred Language          0 non-null      float64
 11  State Issue                 11632 non-null  bool   
 12  Parent Request              1 non-null      float64
 13  Closed When Created        

#### To do: drop columns that don't contain useful data:  
* System of Origin ID  
* Contact Type  
* Preferred Language  
* Parent Request  

In [20]:
# clean up city names
covid_only['City'].value_counts()

NASHVILLE         8627
MADISON            582
ANTIOCH            470
HERMITAGE          431
GOODLETTSVILLE     207
OLD HICKORY        200
BRENTWOOD           50
JOELTON             43
WHITES CREEK        27
LAVERGNE             9
NOLENSVILLE          5
Nashville            1
LA VERGNE            1
Name: City, dtype: int64

In [21]:
# 
covid_only['Status'].value_counts()

Closed    11611
New          21
Name: Status, dtype: int64